In [3]:
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
import numpy as np

In [5]:
from keras.applications.inception_v3 import InceptionV3, decode_predictions

In [7]:
from keras import backend as K

In [8]:
iv3 = InceptionV3()

96116736/96112376 [==============================] - 35s 0us/step


In [10]:
iv3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
from keras.preprocessing import image

In [21]:
x = image.img_to_array(image.load_img("cat.jpeg", target_size=(299,299)))

x.shape

(299, 299, 3)

In [22]:
# reshape the image from 0 ,255 to -1, 1
x /= 255
x -= 0.5
x *= 2

In [23]:
# add one dimention to array, because this is necessary to pass the image to the model
x = x.reshape([1, x.shape[0], x.shape[1], x.shape[2]])

x.shape

(1, 299, 299, 3)

In [24]:
y = iv3.predict(x)

In [28]:
# 1000 is the number of labels in the output

y.shape

(1, 1000)

In [29]:
decode_predictions(y)  #show the name of the most probably label for the prediction

40960/35363 [==================================] - 0s 2us/step


[[('n02123045', 'tabby', 0.4070574),
  ('n02124075', 'Egyptian_cat', 0.38354224),
  ('n02123159', 'tiger_cat', 0.09202612),
  ('n02127052', 'lynx', 0.051025476),
  ('n04553703', 'washbasin', 0.001063339)]]

In [31]:
x = image.img_to_array(image.load_img("sword.jpeg", target_size=(299,299)))

# reshape the image from 0 ,255 to -1, 1
x /= 255
x -= 0.5
x *= 2

# add one dimention to array, because this is necessary to pass the image to the model
x = x.reshape([1, x.shape[0], x.shape[1], x.shape[2]])

y = iv3.predict(x)

decode_predictions(y)

[[('n04141327', 'scabbard', 0.93252605),
  ('n03658185', 'letter_opener', 0.041288015),
  ('n02783161', 'ballpoint', 0.00966901),
  ('n04192698', 'shield', 0.00040614945),
  ('n03000247', 'chain_mail', 0.0003875296)]]

<b>--------------------</b>
<b>adversarial attack</b>
<b>--------------------</b>

In [35]:
inp_layer = iv3.layers[0].input # get the input point of the first layer
out_layer = iv3.layers[-1].output # get the output point of the last layer

print(inp_layer)
print(out_layer)

Tensor("input_1:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("predictions/Softmax:0", shape=(?, 1000), dtype=float32)


In [39]:
target_class = 951 # class that we want to use to cheat the neurla network

loss = out_layer[0, target_class] # we want to use target_class to maximise the cost

# gradient 

grad = K.gradients(loss, inp_layer)[0] # calculates the gradient graph between loss and inp_layer


optimizer_gradient = K.function([inp_layer, K.learning_phase()], [grad, loss])

In [43]:
cost = 0.0


x = image.img_to_array(image.load_img("cat.jpeg", target_size=(299,299)))

# reshape the image from 0 ,255 to -1, 1
x /= 255
x -= 0.5
x *= 2

# add one dimention to array, because this is necessary to pass the image to the model
x = x.reshape([1, x.shape[0], x.shape[1], x.shape[2]])

adv = np.copy(x)

In [44]:
while cost < 0.95:
    gr, cost = optimizer_gradient([adv, 0])
    
    adv += gr
    
    print("target cost:", cost)

target cost: 7.050022e-05
target cost: 7.065126e-05
target cost: 7.083468e-05
target cost: 7.105031e-05
target cost: 7.127559e-05
target cost: 7.1500115e-05
target cost: 7.172772e-05
target cost: 7.1955794e-05
target cost: 7.218768e-05
target cost: 7.242011e-05
target cost: 7.265149e-05
target cost: 7.288253e-05
target cost: 7.311661e-05
target cost: 7.335651e-05
target cost: 7.359855e-05
target cost: 7.3848125e-05
target cost: 7.4097195e-05
target cost: 7.43434e-05
target cost: 7.458862e-05
target cost: 7.48354e-05
target cost: 7.508031e-05
target cost: 7.532438e-05
target cost: 7.5568685e-05
target cost: 7.5813376e-05
target cost: 7.605858e-05
target cost: 7.630573e-05
target cost: 7.655862e-05
target cost: 7.681216e-05
target cost: 7.7064025e-05
target cost: 7.731366e-05
target cost: 7.7560144e-05
target cost: 7.780709e-05
target cost: 7.805545e-05
target cost: 7.830558e-05
target cost: 7.8560624e-05
target cost: 7.8816716e-05
target cost: 7.9072546e-05
target cost: 7.932969e-05
tar

KeyboardInterrupt: 